## Sampling large Datasets
In data processing, a great deal of computing involves analysing large amounts of text mixed with numerical data.  This is what Spark is particularly suited for. Sampling is an essential pre-processing for machine leanring for proof of concept

### Amazon dataset
The file Amazon_xx.inter comprising the ratings of users over the items.
Each record/line in the file has the following fields: user_id, item_id, rating and timestamp.

* user_id: the id of the users and its type is token. 
* item_id: the id of the items and its type is token.
* rating: the rating of the users over the item, and its type is float.
* timestamp: the UNIX time of the interaction, and its type is float.

The file Amazon_xx.item comprising the attributes of the items.
Each record/line in the file has the following fields: item_id, title, price, sales_type, sales_rank, brand, categories
 
* item_id: the id of the item and its type is token.
* title: the title of the item, and its type is token.
* price: the price of the item, and its type is float.
* sales_type: the type sales rank in, and its type is token. 
* sales_rank: sales rank, and its type is float.
* brand: the brand name of the item, and its type is token.
* categories: the categories of the item, and its type is token_seq.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Amazon Sampling").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/05 20:14:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Run the code in this box, and then continue running the boxes (in order) until the end of lesson.
# import pyspark
# conf = pyspark.SparkConf().setAppName("Amazon Sampling")

# try:
#     sc = pyspark.SparkContext(conf=conf)
# except:
#     print("Warning : a SparkContext already exists.")

# readin the top5 samples to get a glimpse of data
movietv5_inter_path = "./Amazon/5Lines/Amazon_Movies_and_TV.inter"
movietv5_item_path = "./Amazon/5Lines/Amazon_Movies_and_TV.item"

# read from file into dataframe
df_movietv5_inter= spark.read.option("delimiter",'\t').option("header", True).csv(movietv5_inter_path)
df_movietv5_inter.show()

df_movietv5_item= spark.read.option("delimiter",'\t').option("header", True).csv(movietv5_item_path)
df_movietv5_item.show()

+--------------+-------------+------------+---------------+
| user_id:token|item_id:token|rating:float|timestamp:float|
+--------------+-------------+------------+---------------+
|A3478QRKQDOPQ2|   0001527665|         5.0|     1362960000|
|A2VHSG6TZHU1OB|   0001527665|         5.0|     1361145600|
|A23EJWOW1TLENE|   0001527665|         5.0|     1358380800|
|A1KM9FNEJ8Q171|   0001527665|         5.0|     1357776000|
|A38LY2SSHVHRYB|   0001527665|         4.0|     1356480000|
+--------------+-------------+------------+---------------+

+-------------+--------------------+--------------------+-----------+----------------+----------------+
|item_id:token|         title:token|categories:token_seq|brand:token|sales_type:token|sales_rank:float|
+-------------+--------------------+--------------------+-----------+----------------+----------------+
|   0000695009|Understanding Sei...|            'Movies'|       null|     Movies & TV|        886503.0|
|   0000791156|Spirit Led&mdash;...|       


## Data Analysis
* uniqueness
* completness 
* lingage, concentration and sparsity

In [29]:
movietv_item_path = "./Amazon/Amazon_Movies_and_TV/Amazon_Movies_and_TV.item"
# read from file into dataframe
df_movietv_item= spark.read.option("delimiter",'\t').option("header", True).csv(movietv_item_path)

print('num of df_movietv_item=',df_movietv_item.count())

# check the uniqueness of key, i.e. item_id
print("Number of disintict item_id:", df_movietv_item.select('item_id:token').distinct().count())

# check the completeness of each column
from pyspark.sql.functions import col, when, count
df_movietv_item.select([count(when(col(c).isNotNull() , c)).alias(c) for c in df_movietv_item.columns]).show()

num of df_movietv_item= 181839
+-------------+-----------+--------------------+-----------+----------------+----------------+
|item_id:token|title:token|categories:token_seq|brand:token|sales_type:token|sales_rank:float|
+-------------+-----------+--------------------+-----------+----------------+----------------+
|       181839|     181781|              177658|     121325|          180003|          179923|
+-------------+-----------+--------------------+-----------+----------------+----------------+



181839

In [27]:
# show null valus samples 
from pyspark.sql.functions import isnull

df_movietv_item.filter(col("title:token").isNull() ).show(5)

df_movietv_item.filter(col("categories:token_seq").isNull()).show(5)

df_movietv_item.filter(col("sales_type:token").isNull()).show(5)

df_movietv_item.filter(col("sales_rank:float").isNull()).show(5)

+-------------+-----------+--------------------+-----------+----------------+----------------+
|item_id:token|title:token|categories:token_seq|brand:token|sales_type:token|sales_rank:float|
+-------------+-----------+--------------------+-----------+----------------+----------------+
|   B00009KM13|       null|            'Movies'|       null|     Movies & TV|        574484.0|
|   B00095L686|       null|            'Movies'|       null|     Movies & TV|        944715.0|
|   B0009UZ4U2|       null|            'Movies'|       null|     Movies & TV|        742364.0|
|   B000M2DMM4|       null|            'Movies'|       null|     Movies & TV|        318541.0|
|   B000TJIUL8|       null|            'Movies'|       null|     Movies & TV|        406440.0|
+-------------+-----------+--------------------+-----------+----------------+----------------+
only showing top 5 rows

+-------------+--------------------+--------------------+--------------+----------------+----------------+
|item_id:toke

In [39]:
movietv_inter_path = "./Amazon/Amazon_Movies_and_TV/Amazon_Movies_and_TV.inter"
# read from file into dataframe
df_movietv_inter= spark.read.option("delimiter",'\t').option("header", True).csv(movietv_inter_path)
print("num of df_movietv_inter=",df_movietv_inter.count(),'\n')

# check the uniqueness of user-item pair
df_movietv_inter_dist=df_movietv_inter.dropDuplicates(["user_id:token","item_id:token"])
print("df_movietv_inter_dist:", df_movietv_inter_dist.count())
print("duplicate interaction num:",df_movietv_inter.count() - df_movietv_inter_dist.count())

# check the completeness of each column
from pyspark.sql.functions import col, when, count
df_movietv_inter_dist.select([count(when(col(c).isNotNull() , c)).alias(c) for c in df_movietv_inter_dist.columns]).show()

num of df_movietv_inter= 8765568 



df_movietv_inter_dist: 8506849


duplicate interaction num: 258719


+-------------+-------------+------------+---------------+
|user_id:token|item_id:token|rating:float|timestamp:float|
+-------------+-------------+------------+---------------+
|      8765568|      8765568|     8765568|        8765568|
+-------------+-------------+------------+---------------+



### Linkage of item and (user-item) interaction
* find least interacted items
* find most interacted(hottest) items

In [41]:
# find the item never interacted
df_movietv_item.join(df_movietv_inter_dist,df_movietv_item["item_id:token"] ==  df_movietv_inter_dist["item_id:token"],"leftanti") \
   .show(truncate=False)

+-------------+------------------------------------------------------------------------------------+--------------------------------------------------+----------------------------------------------------------+----------------+----------------+
|item_id:token|title:token                                                                         |categories:token_seq                              |brand:token                                               |sales_type:token|sales_rank:float|
+-------------+------------------------------------------------------------------------------------+--------------------------------------------------+----------------------------------------------------------+----------------+----------------+
|B00KOD9842   |The Super Chief The Whole Story [DVD] [2008]                                        |'Genre for Featured Categories'                   |Santa Fe                                                  |Movies & TV     |188262.0        |
|B001B48RHM   |THE C

In [ ]:
# find the item few interacted
df_inner_join = df_movietv_item.join(df_movietv_inter_dist,df_movietv_item["item_id:token"] ==  df_movietv_inter_dist["item_id:token"],"inner")

# show the histogram of interaction rate of each item


## Sampling
Stratified sampling based on hotness(interaction rate) of items

In [ ]:
# bining based on the histogram

# sampling by bsed on  interaction rate of items

## Sammary
Spark is a powerful and efficient tool to handle sample on large scale of data. 
* flexible and powerful functionality
* runs super fast even on my laptop
* easy to apply to similar datasets(Amazon have dataset of different categories), I only focused on one categoy this time. 